In [2]:
#!pip3 install ktrain

In [3]:

import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [4]:
data = pd.read_excel('politeness_finalset_data.xlsx')

In [5]:
data.fillna(0, inplace=True)

In [6]:
X_train = data.comment.tolist()
y_train=data['final_agreed_rating'].values
y_train=y_train.tolist()

In [7]:
class_names = ['polite', 'impolite', 'neutral']
print(data.final_agreed_rating.value_counts())


neutral     322
polite      220
impolite     46
0             1
Name: final_agreed_rating, dtype: int64


In [8]:
encoding = {
    'impolite': 0,
    'neutral': 1,
    'polite': 2,
    0: 2,
}

In [9]:
y_train = [encoding[x] for x in y_train]


In [10]:
##splitting the data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_new, y_test= train_test_split(X_train, y_train, test_size=0.1, random_state=125)

#x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size=0.21, random_state=125)

In [11]:
x_train, x_val, y_train, y_val= train_test_split(x_train,  y_train_new, test_size=0.21, random_state=125)

In [12]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [13]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 350
done.


In [14]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [15]:
import tensorflow as tf
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

In [17]:
learner.fit_onecycle(2e-5, 10, callbacks=[callbacks])




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
70/70 [==============================] - 484s 7s/step - loss: 1.0742 - accuracy: 0.3876 - val_loss: 0.8758 - val_accuracy: 0.5085
Epoch 2/10
70/70 [==============================] - 462s 7s/step - loss: 0.8241 - accuracy: 0.6603 - val_loss: 0.7959 - val_accuracy: 0.6271
Epoch 3/10
70/70 [==============================] - 464s 7s/step - loss: 0.5822 - accuracy: 0.7560 - val_loss: 0.8037 - val_accuracy: 0.6780


In [18]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['polite', 'impolite', 'neutral']

In [33]:
import time 
start_time = time.time() 
#prediction=[]
print(x_val[0])
prediction = (predictor.predict([x_val[6]]))


@ItaloKnox still doesn't work what OS patch version and graphics version are you using? I made a full reinstall.. but it still doesn't work.. i'll just try deleting even my saves. Thanks for the quick response. I hope I can resolve this.. much better experience when using this mod."


In [34]:
print(prediction)

['impolite']


In [39]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [41]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)


              precision    recall  f1-score   support

      polite       1.00      0.11      0.20         9
    impolite       0.75      0.73      0.74        67
     neutral       0.61      0.78      0.68        36

    accuracy                           0.70       112
   macro avg       0.79      0.54      0.54       112
weighted avg       0.73      0.70      0.68       112



array([[ 1,  8,  0],
       [ 0, 49, 18],
       [ 0,  8, 28]])

In [42]:
predictor.save("models/bert_model_polite")

In [43]:

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['polite', 'impolite', 'neutral']

In [44]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: impolite (0.29)
